In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
def keep_data_tables(filePath,sheetName1,sheetName2):
    workbook = openpyxl.load_workbook(filePath)    
    table_names = workbook.sheetnames

    tables_to_keep = [sheetName1,sheetName2]  
    for table_name in table_names:
        if table_name not in tables_to_keep:
            del workbook[table_name]

    
    workbook.save(filePath)    
    workbook.close()

In [3]:
def remove_pictures(filePath):    
    workbook = openpyxl.load_workbook(filePath)    
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        for img in sheet._images:
            sheet._images.remove(img)    
    workbook.save(filePath)

In [4]:
def delete_rows_from_first_sheet(filePath, sheetName, rows_to_delete):
    df = pd.read_excel(filePath, sheet_name=sheetName)

    df = df.drop(rows_to_delete)

    with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheetName, index=False)

In [5]:
def delete_rows_from_second_sheet(filePath, sheetName, rows_to_delete):
    df = pd.read_excel(filePath, sheet_name=sheetName)
    df = df.drop(rows_to_delete)

    with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheetName, index=False)

In [6]:
def delete_non_data_rows(filePath, sheetName1, sheetName2, rows_to_delete1, rows_to_delete2):
    delete_rows_from_first_sheet(filePath,sheetName1, rows_to_delete1)
    delete_rows_from_second_sheet(filePath,sheetName2, rows_to_delete2)

In [7]:
def delete_hospital_name_column(filePath,sheetName):
    df = pd.read_excel(filePath, sheet_name=sheetName)
    df.drop(df.columns[1], axis=1, inplace=True)

    with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheetName, index=False)

In [8]:
def convert_numerals_benTOeng(filePath, sheet_name):
    bengali_to_english = {
        '০': '0',
        '১': '1',
        '২': '2',
        '৩': '3',
        '৪': '4',
        '৫': '5',
        '৬': '6',
        '৭': '7',
        '৮': '8',
        '৯': '9',
    }

    workbook = openpyxl.load_workbook(filePath)

   
    worksheet = workbook[sheet_name]


    for row in worksheet.iter_rows():
        for cell in row:
            if cell is not None and isinstance(cell.value, str):
                cell.value = ''.join(bengali_to_english.get(char, char) for char in cell.value)

 
    workbook.save(filePath)  


In [9]:
def merge_two_tables(filePath, sheet1, sheet2):
    df1 = pd.read_excel(filePath,sheet1)
    df2 = pd.read_excel(filePath,sheet2)
    merged_df = pd.concat([df1, df2], axis=0, ignore_index=True)
    with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        merged_df.to_excel(writer, sheet_name='MergedSheet', index=False)   

In [10]:
def preprocessing(filePath, sheetName1, sheetName2, rows_to_delete1, rows_to_delete2):
    keep_data_tables(filePath,sheetName1,sheetName2)
    remove_pictures(filePath)
    delete_non_data_rows(filePath, sheetName1, sheetName2, rows_to_delete1, rows_to_delete2)
    # delete_hospital_name_column(filePath, sheetName1)
    # delete_hospital_name_column(filePath, sheetName2)
    convert_numerals_benTOeng(filePath,sheetName1)
    convert_numerals_benTOeng(filePath,sheetName2)
    # merge_two_tables()

In [11]:
import os
import glob

In [12]:
directory_path = '2023-06/s'
excel_files = glob.glob(os.path.join(directory_path, '*.xlsx'))
r1 = [0, 1, 2]
r2 = [0, 1]
sheetName1 = "Table 17"
sheetName2 = "Table 19"
for filePath in excel_files:
    preprocessing(filePath,sheetName1,sheetName2,r1,r2)
    print(filePath + " completed")